In [11]:
#pip install langchain langchain-experimental openai pymysql

In [12]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

#initialize the llm
OPENAI_API_KEY="sk-proj-xb4t01kHNIFwlXU_70fHGaVbC5uURYMh5U6GzZekXbPe1DphuTzzT3BlbkFJ4wpq6Q2l5yWB3JUlBBtQOV5YwxeLW1ZDL5_AR1G0-iJbPGp09_pAA"
llm=ChatOpenAI(temperature=0,openai_api_key=OPENAI_API_KEY)


In [13]:
from urllib.parse import quote_plus
host='localhost'
port=3306
username='root'
password=quote_plus('Mysql@1234')
database_schema='movies'
mysql_uri=f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
db=SQLDatabase.from_uri(mysql_uri, include_tables=['telugumovies_dataset'],sample_rows_in_table_info=2)
db_chain=SQLDatabaseChain.from_llm(llm,db,verbose=True)

In [41]:
def retrieve_from_db(query:str)-> str:
    db_context=db_chain(query)
    db_context=db_context['result'].strip()
    return db_context

def generate(query):
    db_context=retrieve_from_db(query)
    system_message="""you are a movie database expert specializing in telugu cinema.
    your task is to answer users' questions by providing relevant information from a database of Telugu movies.
    The database contains the following information for each movie:
    -Name of the movie
    -Year fo release
    -Certificate given by the censor board
    -Movie genre
    -A brief description or plot of the movie
    -Duration in minutes
    -IMDb Rating
    -Number of people who rated the movie
    
    Example:
    
    Input:
    Which Telugu movies released in 2021 have the highest IMDb ratings?
    
    Context:
    The Telugu movies released in 2021 with the highest IMDb ratings are :
    1. Movie A - EMDb rating: 8.5
    2. Movie B - EMDb rating: 8.3
    
    Output:
    The highest-rated Telugu movies released in 2021 are Movie A (IMDb Rating: 8.5) and Movie B(IMDb Rating: 8.3).
    """
    human_qry_template=HumanMessagePromptTemplate.from_template(
        """Input:
        {human_input}
        Context:
        {db_context}
        Output:
    
        """
    )
    messages=[
        SystemMessage(content=system_message),
        human_qry_template.format(human_input=query,db_context=db_context)
    ]
    response=llm(messages).content
    return response
generate("how many movies are thee?")   




> Entering new SQLDatabaseChain chain...
how many movies are there?
SQLQuery:SELECT COUNT(`ID`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1350,)]
Answer:There are 2 movies.
> Finished chain.


'There are 2 movies in the database.'

In [39]:
# db_context=retrieve_from_db(query)
generate("how many movies IMDb rating above 3?")
# db_context=db_chain(query)
# response=llm(messages).content



> Entering new SQLDatabaseChain chain...
how many movies IMDb rating above 3?
SQLQuery:SELECT COUNT(`ID`) AS `Num_Movies_Above_3`
FROM telugumovies_dataset
WHERE `Rating` > 3;
SQLResult: [(1327,)]
Answer:There are 1327 movies with an IMDb rating above 3.
> Finished chain.


'There are 1327 movies with an IMDb rating above 3.'

In [43]:
generate("what is the best movie?")



> Entering new SQLDatabaseChain chain...
what is the best movie?
SQLQuery:SELECT `Movie`, `Rating`, `No.of.Ratings` 
FROM telugumovies_dataset 
ORDER BY `Rating` DESC, `No.of.Ratings` DESC 
LIMIT 1;
SQLResult: [('Pichhodu', 9.5, 605)]
Answer:The best movie is "Pichhodu" with a rating of 9.5 and 605 number of ratings.
> Finished chain.


'The best Telugu movie is "Pichhodu" with an IMDb rating of 9.5 and 605 number of ratings.'

In [45]:
generate("what are different values in the column in IMDb rating?")



> Entering new SQLDatabaseChain chain...
what are different values in the column in IMDb rating?
SQLQuery:SELECT DISTINCT `Rating` FROM telugumovies_dataset;
SQLResult: [(8.1,), (8.2,), (5.4,), (4.6,), (6.5,), (7.7,), (5.2,), (8.4,), (7.6,), (7.1,), (7.4,), (7.9,), (6.8,), (6.2,), (5.9,), (7.5,), (5.6,), (8.5,), (6.6,), (3.3,), (6.7,), (6.4,), (7.2,), (8.0,), (7.3,), (6.1,), (7.8,), (4.4,), (8.6,), (6.3,), (5.8,), (7.0,), (8.3,), (5.0,), (9.2,), (9.0,), (6.9,), (8.7,), (6.0,), (5.5,), (4.3,), (8.8,), (5.7,), (4.5,), (8.9,), (3.2,), (4.8,), (4.2,), (5.1,), (5.3,), (4.9,), (3.5,), (4.7,), (2.4,), (9.5,), (3.7,), (3.6,), (2.5,), (2.1,), (3.1,), (2.9,), (2.0,), (3.8,), (3.4,), (2.7,), (4.0,), (3.9,), (3.0,), (4.1,), (1.6,), (1.3,), (1.2,), (1.1,), (1.7,), (1.4,), (1.0,)]
Answer:The different values in the IMDb rating column are: 8.1, 8.2, 5.4, 4.6, 6.5, 7.7, 5.2, 8.4, 7.6, 7.1, 7.4, 7.9, 6.8, 6.2, 5.9, 7.5, 5.6, 8.5, 6.6, 3.3, 6.7, 6.4, 7.2, 8.0, 7.3, 6.1, 7.8, 4.4, 8.6, 6.3, 5.8, 7.0, 8

'The different values in the IMDb rating column are: 8.1, 8.2, 5.4, 4.6, 6.5, 7.7, 5.2, 8.4, 7.6, 7.1, 7.4, 7.9, 6.8, 6.2, 5.9, 7.5, 5.6, 8.5, 6.6, 3.3, 6.7, 6.4, 7.2, 8.0, 7.3, 6.1, 7.8, 4.4, 8.6, 6.3, 5.8, 7.0, 8.3, 5.0, 9.2, 9.0, 6.9, 8.7, 6.0, 5.5, 4.3, 8.8, 5.7, 4.5, 8.9, 3.2, 4.8, 4.2, 5.1, 5.3, 4.9, 3.5, 4.7, 2.4, 9.5, 3.7, 3.6, 2.5, 2.1, 3.1, 2.9, 2.0, 3.8, 3.4, 2.7, 4.0, 3.9, 3.0, 4.1, 1.6, 1.3, 1.2, 1.1, 1.7, 1.4, 1.0.'

In [47]:
generate("what are range of values in the column in IMDb rating?")



> Entering new SQLDatabaseChain chain...
what are range of values in the column in IMDb rating?
SQLQuery:SELECT MIN(`Rating`) AS Min_Rating, MAX(`Rating`) AS Max_Rating
FROM telugumovies_dataset;
SQLResult: [(1.0, 9.5)]
Answer:The range of values in the IMDb rating column is from 1.0 to 9.5.
> Finished chain.


'The range of values in the IMDb rating column is from 1.0 to 9.5.'

In [49]:
generate("what is the highest mountain inthe world")



> Entering new SQLDatabaseChain chain...
what is the highest mountain inthe world
SQLQuery:SELECT `Movie`, `Rating` 
FROM telugumovies_dataset 
ORDER BY `Rating` DESC 
LIMIT 1;
SQLResult: [('Pichhodu', 9.5)]
Answer:The highest-rated movie in the dataset is "Pichhodu" with a rating of 9.5.
> Finished chain.


"I'm sorry, I specialize in providing information about Telugu movies. If you have any questions related to Telugu cinema, feel free to ask!"

In [51]:
generate("how many movies are three?")



> Entering new SQLDatabaseChain chain...
how many movies are three?
SQLQuery:SELECT COUNT(`ID`) AS `TotalMovies` FROM telugumovies_dataset;
SQLResult: [(1350,)]
Answer:There are 1350 movies in the dataset.
> Finished chain.


'There are 1350 movies in the dataset.'

In [53]:
generate("how many movies are there?")



> Entering new SQLDatabaseChain chain...
how many movies are there?
SQLQuery:SELECT COUNT(`ID`) AS TotalMovies FROM telugumovies_dataset;
SQLResult: [(1350,)]
Answer:There are 1350 movies in the dataset.
> Finished chain.


'There are 1350 movies in the dataset.'